In [ ]:
import warnings
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window
from delta.tables import DeltaTable

warnings.filterwarnings("ignore", category=FutureWarning)

# Create SparkSession
spark = SparkSession.builder.appName("DeltaSession") \
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
            .getOrCreate()

## Create Table

In [ ]:
(DeltaTable
    .createOrReplace(spark)
    .tableName("03updateschema")
    .addColumn("id", "INT")
    .addColumn("name", "STRING")
    .addColumn("age", "STRING")
    .addColumn("city", "STRING", comment = "cityWorld")
    .property("description", "table with people data")
    .property("abd", "123")
    .partitionedBy("id")
    .execute()
)

In [ ]:
spark.sql("SELECT * FROM 03updateschema").toPandas()

In [ ]:
spark.sql("""
INSERT INTO 03updateschema (id, name, age, city)
VALUES (1, 'Marcelo', 5, 'Sao Paulo')
""")

In [ ]:
spark.sql("SELECT * FROM 03updateschema").toPandas()

In [ ]:
spark.sql("DESCRIBE 03updateschema").toPandas()

## Enable Column Mapping

In [ ]:
spark.sql("""
ALTER TABLE 03updateschema SET TBLPROPERTIES (
   'delta.minReaderVersion' = '2',
   'delta.minWriterVersion' = '5',
   'delta.columnMapping.mode' = 'name'
)
""")

## Add Column

In [ ]:
spark.sql("ALTER TABLE 03updateschema ADD COLUMN address STRING COMMENT 'endereço da pessoa'")

In [ ]:
spark.sql("DESCRIBE 03updateschema").toPandas()

In [ ]:
spark.sql("SELECT * FROM 03updateschema").toPandas()

## Drop Column

In [ ]:
spark.sql("ALTER TABLE 03updateschema DROP COLUMN address")

In [ ]:
spark.sql("DESCRIBE 03updateschema").toPandas()

In [ ]:
spark.sql("SELECT * FROM 03updateschema").toPandas()

## Rename Column

In [ ]:
spark.sql("ALTER TABLE 03updateschema RENAME COLUMN city TO city2")

In [ ]:
spark.sql("DESCRIBE 03updateschema").toPandas()

In [ ]:
spark.sql("SELECT * FROM 03updateschema").toPandas()

## Column Comment

In [ ]:
spark.sql("ALTER TABLE 03updateschema ALTER COLUMN city2 COMMENT 'cidade de residencia'")

In [ ]:
spark.sql("DESCRIBE 03updateschema").toPandas()

## Column Type

In [ ]:
(spark.read
     .format("delta")
     .table("03updateschema")
     .withColumn("age", F.col("age").cast("int"))
     .write
     .mode("overwrite")
     .option("overwriteSchema", "true")
     .saveAsTable("03updateschema")
)